In [13]:
import pandas as pd
import json

# Populate cards_database.json with external database

## Load cards_datatabase.json

In [14]:
def json_to_df_and_excel():
    # Load the JSON file
    with open(r"C:\Users\RubénLopez\Documents\mtg_decklists_analyzer\cards_database.json", "r") as file:
        cards_database = json.load(file)

    # Flatten the JSON structure
    data = []
    for format, cards in cards_database.items():
        for card_name, attributes in cards.items():
            row = {
                'format': format,
                'card_name': card_name,
                'type': attributes.get('type'),
                'subtype': attributes.get('subtype'),
                'color': attributes.get('color'),
            }
            data.append(row)

    # Convert the data into a DataFrame
    df = pd.DataFrame(data)

    # Export the DataFrame to an Excel file
    df.to_excel('cards_database.xlsx', index=False)

    return df

In [15]:
df_json = json_to_df_and_excel()
df_json

,format,card_name,type,subtype,color
0,Premodern,Mox Diamond,Acceleration,Tier 1,Colorless
1,Premodern,Hydroblast,Hate,Red,Blue
2,Premodern,Blue Elemental Blast,Hate,Red,Blue
3,Premodern,Counterspell,Counter,Tier 1,Blue
4,Premodern,Mana Leak,Counter,Tier 1,Blue
...,...,...,...,...,...
655,Premodern,Energy Flux,unknown,unknown,unknown
656,Premodern,Sindbad,unknown,unknown,unknown
657,Premodern,Auratog,unknown,unknown,unknown
658,Premodern,Hurkyl's Recall,unknown,unknown,unknown


In [16]:
df_json[df_json.card_name=="Attunement"]

,format,card_name,type,subtype,color
638,Premodern,Attunement,unknown,unknown,unknown


## Load external database

In [17]:
df_all_cards = pd.read_csv("output_modified.csv", index_col=0)
df_all_cards.drop_duplicates(subset="cardName", keep="first", inplace=True)
df_all_cards.head()

,cardName,superType,cardType,subType,typeNum,manaCost,convertedMana,cardSet,rarity,rules,color
0,"""Ach! Hans, Run!""",NaN,Enchantment,NaN,NaN,2RRGG,6,Unhinged,Rare,"At the beginning of your upkeep, you may say ""...",Multicolored
1,"""Rumors of My Death . . .""",NaN,Enchantment,NaN,NaN,2B,3,Unstable,Uncommon,"3B, Exile a permanent you control with a Leagu...",Black
2,_____,NaN,Creature,Shapeshifter,1/1,1U,2,Unhinged,Uncommon,1: This card's name becomes the card name of y...,Blue
3,A Good Thing,NaN,Enchantment,NaN,NaN,4WB,6,Mystery Booster,Common,Spells and abilities you control can't destroy...,Multicolored
4,Abandon Hope,NaN,Sorcery,NaN,NaN,X1B,2,Tempest,Uncommon,"As an additional cost to cast this spell, disc...",Black


In [18]:
df_all_cards[df_all_cards["cardName"] == "Attunement"]

,cardName,superType,cardType,subType,typeNum,manaCost,convertedMana,cardSet,rarity,rules,color
1861,Attunement,NaN,Enchantment,NaN,NaN,2U,3,Urza's Saga,Rare,Return Attunement to its owner's hand: Draw th...,Blue


## Populate cards_database with external database

In [19]:
def populate_column(column_cards_db, column_ext, df_cards_db, df_ext):
    # Set the card names as the index
    df_cards_db.set_index('card_name', inplace=True)
    df_ext.set_index('cardName', inplace=True)

    # Replace 'Unknown' colors in df_json with colors from df_all_cards
    df_cards_db[column_cards_db] = df_cards_db[column_cards_db].where(df_cards_db[column_cards_db] != 'unknown', df_cards_db.index.map(df_ext[column_ext]))

    # Reset the index
    df_cards_db.reset_index(inplace=True)
    df_ext.reset_index(inplace=True)

    return df_cards_db

In [20]:
df_json = populate_column("color", "color", df_json, df_all_cards)
df_json = populate_column("type", "cardType", df_json, df_all_cards)

In [21]:
df_json = df_json[['format', 'card_name', 'type', 'subtype', 'color']]
df_json

,format,card_name,type,subtype,color
0,Premodern,Mox Diamond,Acceleration,Tier 1,Colorless
1,Premodern,Hydroblast,Hate,Red,Blue
2,Premodern,Blue Elemental Blast,Hate,Red,Blue
3,Premodern,Counterspell,Counter,Tier 1,Blue
4,Premodern,Mana Leak,Counter,Tier 1,Blue
...,...,...,...,...,...
655,Premodern,Energy Flux,Enchantment,unknown,Blue
656,Premodern,Sindbad,Creature,unknown,Blue
657,Premodern,Auratog,Creature,unknown,White
658,Premodern,Hurkyl's Recall,Instant,unknown,Blue


In [22]:
df_json[df_json["card_name"]=="Attunement"]

,format,card_name,type,subtype,color
638,Premodern,Attunement,Enchantment,unknown,Blue


# Update cards_database.json

In [23]:
def df_to_json(df_json):
    cards_database = {}
    for row in df_json.iterrows():
        row_list = row[1].values
        if row_list[0] not in cards_database:
            cards_database[row_list[0]] = {}
        if row_list[1] not in cards_database[row_list[0]]:
            cards_database[row_list[0]][row_list[1]] = {}
        cards_database[row_list[0]][row_list[1]]["type"] = row_list[2]
        cards_database[row_list[0]][row_list[1]]["subtype"] = row_list[3]
        cards_database[row_list[0]][row_list[1]]["color"] = row_list[4]
    # Save cards_database dict
    with open("cards_database.json", "w") as file:
        json.dump(cards_database, file)

In [24]:
df_to_json(df_json)